# 状態方程式の線形化の数式処理

吉田勝俊（宇都宮大学）

### 参考情報
- [Welcome to SymPy’s documentation! &#8212; SymPy 1.10.1 documentation](https://docs.sympy.org/) （本家）
- [SymPy による数式処理とグラフ作成 - 弘前大学 Home Sweet Home](https://home.hirosaki-u.ac.jp/jupyter/sympy/)

In [ ]:
import sympy as sym #数式処理ライブラリ

## ◯オイラー・ラグランジュ方程式

### 定義：

$\displaystyle
\frac{d}{dt}\left(\frac{\partial L}{\partial \dot q_i}\right)
-\frac{\partial L}{\partial q_i}
+\frac{\partial D}{\partial \dot q_i}
-Q_i
= 0, \quad i=1,2,\cdots
$

In [ ]:
def Derive_ELE(system):
    '''
    オイラー・ラグランジュ方程式（ELE: Euler-Lagrange equation）
    「左辺＝０」の左辺を導出する関数
    
    def system():    
        q: 一般化座標（ベクトル）
        L: ラグランジュ関数（スカラ）
        D: 散逸関数（スカラ）
        Q: 一般化力（ベクトル）
        u: 外部入力（ベクトル）※線形化で用いる
        
        return (t, q, L, D, Q, u)
    '''
    t, q, L, D, Q, u = system()       #一般化座標, ラグランジアン, 散逸関数, 一般化力
    dqdt             = sym.diff(q, t) #一般化座標の時間微分
    dim              = len(q)         #一般化座標の次元
    
    ### ラグランジュの運動方程式 ###
    ELE = sym.Matrix([
        sym.simplify(
            sym.diff( sym.diff(L, dqdt[i]), t )
            - sym.diff(L, q[i])
            + sym.diff(D, dqdt[i])
            - (Q[i] if Q is not None else 0) #Q が無ければ 0
        )
        for i in range(dim)
    ])

    return (ELE, q, t, u)

## ◯1階化

In [ ]:
def FirstOrdered(ELE):
    '''
    運動方程式を1階化するユーザ関数
    - ELE ... Derive_ELE(system)の出力
    '''
    EOMq, q, t, u = ELE

    ### 状態ベクトルの生成 ###
    # 実体
    xq = sym.Matrix(
        [qi for qi in q] 
        + #リストの + は連結
        [sym.diff(qi, t) for qi in q]
    )

    # 通し番号
    x = sym.Matrix([
        sym.Function(r'x_' + str(i+1), real=True)(t)
        for i in range(len(xq))
    ])

    # 加速度（通し番号）
    accel = sym.Matrix([
        sym.diff(xi, t)
        for xi in x
    ])

    ### 状態ベクトルの生成 ###
    dim = len(q)
    EOMx = []
    for eqn in EOMq:
        
        #運動方程式を通し番号の状態ベクトルで表す
        for i in range(dim):
            eqn = eqn.subs(xq[dim+i], x[dim+i])
            eqn = eqn.subs(xq[i], x[i])
            
        EOMx.append(eqn)

    ### 加速度について解く ###
    sols = sym.solve(EOMx, accel[dim:])
    
    ### 1階化した運動方程式 dx/dt = f(x) の右辺 ###
    fx = sym.Matrix(
        x[dim:] 
        + [sols[a].simplify() for a in accel[dim:]]
    )

    return (fx, x, xq, t, u)

## ◯ 線形化

In [ ]:
def initial_state(x0, x0symb, x):
    '''
    線形化の基準点が未指定なら記号で生成する
    '''
    if x0 is None:
        if len(x) == 1:  #基準点がスカラ（1次元）の場合
            x0 = sym.Matrix([x0symb])
        else:            #基準点がベクトル（多次元）の場合
            x0 = sym.Matrix([
                sym.symbols(x0symb + '_' + str(i+1)) for i in range(len(x))
            ])

    else:
        x0 = sym.Matrix(x0)
    return x0

In [ ]:
def Linearized(first_ordered, x0=None, u0=None):
    '''
    1階化した運動方程式を，x0 の近傍で線形化する
    - first_ordered ... FirstOrdered(ELE) の出力
    '''
    fx, x, xq, t, u = first_ordered

    # 状態ベクトルに関するヤコビ行列
    dfxdx = fx.jacobian(x)          #ヤコビ行列
    x0 = initial_state(x0, 'x0', x) #基準点の準備
    for xi, x0i, in zip(x, x0):     #基準点の代入（状態）
        dfxdx = dfxdx.subs(xi, x0i)

    dfxdx = sym.simplify(dfxdx)
        
    jacobi_x = {'x':dfxdx, 'x0':x0} #(ヤコビ行列，基準点)
    
    # 外部入力に関するヤコビ行列
    if u is not None: #もし外部入力が存在すれば
        dfxdu = fx.jacobian(u)          #ヤコビ行列
        u0 = initial_state(u0, 'u0', u) #基準点の準備
        for ui, u0i, in zip(u, u0):     #基準点の代入（入力）
            dfxdu = dfxdu.subs(ui, u0i)
        for xi, x0i, in zip(x, x0):     #基準点の代入（状態）
            dfxdu = dfxdu.subs(xi, x0i)
            
        dfxdu = sym.simplify(dfxdu)
        jacobi_u = {'u':dfxdu, 'u0':u0} #(ヤコビ行列，基準点)
        
    else:
        jacobi_u = {'u':None, 'u0':None}

    ### 結果（辞書型） ###
    jacobi_matrices = {'xq':xq}      #状態ベクトルの定義
    jacobi_matrices.update(jacobi_u) #xの(ヤコビ行列，基準点)をマージ
    jacobi_matrices.update(jacobi_x) #uの(ヤコビ行列，基準点)をマージ
        
    return jacobi_matrices

### ■単振り子（SP: simple pendulum）

- 一般化座標: $(\theta(t))$

In [ ]:
def system_SP():
    '''
    システムを定義する関数（単振り子）
    '''
    # パラメータ: 
    t, m, l, g = sym.symbols(
        't m l g', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    c = sym.symbols('c', positive=True)
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    theta = q[0]
    xx = l*sym.Matrix([
        sym.sin(theta),
        -sym.cos(theta),
    ])
    
    dxxdt = sym.diff(xx, t) #その時間微分
    
    # 運動エネルギー
    T = (m/2)*dxxdt.dot(dxxdt)
    
    # 位置エネルギー
    h = xx[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
#     D = (c/2)*dxxdt.dot(dxxdt) #粘性減衰
    D = 0 #減衰なし

    # 外部入力（この系は無し）
    u = None

    # 一般化力
    Q = u
    
    return (t, q, L, D, Q, u)

#### 運動方程式の導出結果

In [ ]:
ele_SP = Derive_ELE(system_SP)

for _ in ele_SP: display(_)

#### 1階化の結果

In [ ]:
first_ordered_SP = FirstOrdered(ele_SP)

display(*first_ordered_SP)

#### 線形化の結果

In [ ]:
linearized_SP = Linearized(first_ordered_SP)
for _ in ['xq', 'x', 'x0', 'u', 'u0']:
    display(linearized_SP[_])

出力にある行列により，$\boldsymbol{x}_0 = (x_{01},x_{02})^T$の近傍で線形化した状態方程式は次のように表せます．

$$
\begin{bmatrix}
 \dot \xi_1 \\ \dot \xi_2
\end{bmatrix}
=
\begin{bmatrix}
 0 & 1 \\
 -\frac{g\cos x_{01}}{l} & 0 \\
\end{bmatrix}
\begin{bmatrix}
 \xi_1 \\ \xi_2
\end{bmatrix}
$$

### ■台車型倒立振子（CIP: cart inverted pendulum）

- 一般化座標: $(x(t),\theta(t))$

In [ ]:
def system_CIP():
    '''
    システムを定義する関数 （問題に応じて書き換える）
    ※以下は，台車型倒立振子の例
    '''
    # パラメータ: 
    t, M, m, l, g, c1, c2 = sym.symbols(
        't M m l g c_1 c_2', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    G, S = sym.symbols('G S', positive=True)
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'x', 
            real=True #実数値に制限
        )(t),
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    x, th = q
    xM = sym.Matrix([
        x,
        G
    ])
    xm = sym.Matrix([
        x + l*sym.sin(th),
        l*sym.cos(th) + S,
    ])
    
    dxMdt = sym.diff(xM, t) #その時間微分
    dxmdt = sym.diff(xm, t)
    
    # 運動エネルギー
    T = (M/2)*dxMdt.dot(dxMdt) + (m/2)*dxmdt.dot(dxmdt)
    
    # 位置エネルギー
    h = xm[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
    D = (c1/2)*sym.diff(x,t)**2 + (c2/2)*sym.diff(th,t)**2 #粘性減衰
#     D = 0 #減衰なし

    # 外部入力（この系は１個だけ）
    u = sym.Matrix([
        sym.Function(r'u', real=True)(t)
    ])
     
    # 一般化力（外部入力が第１成分のみに作用）
    Q = sym.Matrix([
        u[0],
        0
    ])
    
    return (t, q, L, D, Q, u)

#### 運動方程式の導出結果

In [ ]:
ele_CIP = Derive_ELE(system_CIP)

for _ in ele_CIP: display(_)

#### 1階化の結果

In [ ]:
first_ordered_CIP = FirstOrdered(ele_CIP)

display(*first_ordered_CIP)

#### 線形化の結果

In [ ]:
linearized_CIP = Linearized(first_ordered_CIP, [0,0,0,0])
for _ in ['xq', 'x', 'x0', 'u', 'u0']:
    display(linearized_CIP[_])

出力にある1番目と4番目の行列により，$\boldsymbol{x}_0 = (0,0)^T$, $\boldsymbol{u}_0 = u_{0}$の近傍で線形化した状態方程式は次のように表せます．

$$
\begin{bmatrix}
 \dot \xi_1 \\ \dot \xi_2 \\ \dot \xi_3 \\ \dot \xi_4
\end{bmatrix}
=
\begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
0 & -\frac{mg}{M}     & -\frac{c_1}{M} & \frac{c_2}{Ml} \\
0 & \frac{(M+m)g}{Ml} &  \frac{c_1}{Ml} & -\frac{c_2(M+m)}{Mml^2} \\
\end{bmatrix}
\begin{bmatrix}
 \xi_1 \\ \xi_2 \\ \xi_3 \\ \xi_4
\end{bmatrix}
+
\begin{bmatrix}
 0 \\ 0 \\ 
 \frac{1}{M} 
 \\
 -\frac{1}{lM} 
\end{bmatrix}
\mu(t)
$$
